## NER - Natasha

Natasha  - питоновская библиотека для извлечения именованных сущностей. Она похоже на Tomita-parser, но в ней все на чистом питоне, с открытым кодом и активно развивается. 

Если быть точнее, то natasha - набор готовых правил для парсера yargy. 

Есть например готовые правила для извлечения персон.

In [1]:
# Установить можно через pip.
from natasha import NamesExtractor

In [2]:
sents = open('sents.txt').read().splitlines()

In [3]:
extractor = NamesExtractor()

In [4]:
# extractor('\n'.join(sents))

Видно, что извлекается не очень хорошо. К тому же в реальной задаче нам потребуется извлекать ещё какие-то аттрибуты сущностей и группировать их в факты. И не обязательно этих типов. Поэтому полезно разобраться с самим парсером yargy.

Документация yargy: https://yargy.readthedocs.io/ru/latest/reference.html

Лучше всего с такими штуками разбираться на практике. Давайте попробуем написать правила для извлечения персон. Каждая персона должна описываться 3 полями - Имя, Фамилия, Отчество. Также у Персоны должны быть атрибуты - должность и место работы. 

За основу возьмем пример из документации:

- Дополните правила в ноутбуке таким образом, чтобы из приведенных ~200 предложений извлекалось больше 50 персон. Можно расширить список профессий, можно добавить атрибут отчества или атрибут места работы (геолокация или организация, которые часто идут **между должностью и самим именем**). 

In [5]:
from yargy import Parser, rule, or_
from yargy.predicates import gram, normalized
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.tokenizer import MorphTokenizer 
from IPython.display import display

Person = fact(
    'Person',
    ['position', 'name', 'place']
)
Name = fact(
    'Name',
    ['first', 'last', 'patronymic']
)


POSITION = morph_pipeline([
    'премьер министр',
    'премьер-министр',
    'министр',
    'вице-премьер',
    'министр финансов',
    'министр обороны',
    'министр иностранных дел',
    'пресс-секретарь',
    'президент'
])


NAME = or_(
    rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    ),
    gram('Patr').interpretation(
        Name.patronymic.inflected()
    ).optional()
).interpretation(
        Name
),
    rule(
    gram('Name').repeatable().interpretation(
        Name.last.inflected())
).interpretation(
        Name
))


PLACE = rule(
        gram('Geox').interpretation(
            Person.place.normalized()
).optional()
)


PERSON = rule(
    POSITION.interpretation(
        Person.position.normalized()
    ),
    PLACE,
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)

parser = Parser(PERSON)

In [6]:
matches = []

for sent in sents:
#     print(sent)
    for match in parser.findall(sent):
        matches.append(match.fact)
#         display(match.fact.name)
#     print('---------------')

In [7]:
len(matches)

80

In [9]:
matches[:3]

[Person(position='премьер-министр',
        name=Name(first='дмитрий',
                  last='медведев',
                  patronymic=None),
        place='россия'), Person(position='премьер-министр',
        name=Name(first=None,
                  last='биньямин нетаньяха',
                  patronymic=None),
        place='израиль'), Person(position='премьер-министр',
        name=Name(first=None,
                  last='нетаньяха',
                  patronymic=None),
        place=None)]

Чтобы проверить какие морфологические тэги приписываются словам, можно использовать MorphTokenizer.

In [10]:
from yargy.tokenizer import MorphTokenizer

In [11]:
tokenizer = MorphTokenizer()

In [12]:
list(tokenizer('премьер-министр России Дмитрий Медведев'))

[MorphToken('премьер',
            [0, 7),
            'RU',
            [Form('премьер', Grams(NOUN,anim,masc,nomn,sing)),
             Form('премьера', Grams(NOUN,femn,gent,inan,plur))]),
 Token('-', [7, 8), 'PUNCT'),
 MorphToken('министр',
            [8, 15),
            'RU',
            [Form('министр', Grams(NOUN,anim,masc,nomn,sing))]),
 MorphToken('России',
            [16, 22),
            'RU',
            [Form('россия', Grams(Geox,NOUN,Sgtm,femn,gent,inan,sing)),
             Form('россия', Grams(Geox,NOUN,Sgtm,femn,inan,loct,sing)),
             Form('россия', Grams(Geox,NOUN,Sgtm,datv,femn,inan,sing))]),
 MorphToken('Дмитрий',
            [23, 30),
            'RU',
            [Form('дмитрий', Grams(NOUN,Name,anim,masc,nomn,sing)),
             Form('дмитрия', Grams(NOUN,Name,anim,femn,gent,plur)),
             Form('дмитрия', Grams(NOUN,Name,accs,anim,femn,plur))]),
 MorphToken('Медведев',
            [31, 39),
            'RU',
            [Form('медведев', Grams(NOUN